In [1]:
import phenopackets.schema.v2 as pps2
from phenopackets.schema.v2.core.individual_pb2 import MALE, FEMALE
from phenopackets.schema.v2.core.pedigree_pb2 import Pedigree
from google.protobuf.json_format import MessageToDict
from google.protobuf.timestamp_pb2 import Timestamp
import json
import datetime

## Phenopackets

### Ontology Class

In [2]:
obese = pps2.OntologyClass(id="NCIT:_C159658", label = "Obese")
tdisease = pps2.OntologyClass(id="NCIT:C3879", label = "Thyroid Gland Medullary Carcinoma")
thyroid = pps2.OntologyClass(id="NCIT:C12400", label = "Thyroid Gland")

### Phenotype & Disease

In [3]:
pobese = pps2.PhenotypicFeature(type = obese)
dthyroid = pps2.Disease(term = tdisease, primary_site=thyroid)

### Metadata

In [4]:
ncit = pps2.Resource(id="ncit",name="NCI Thesaurus OBO Edition",url="http://purl.obolibrary.org/obo/ncit.owl",version="https://github.com/NCI-Thesaurus/thesaurus-obo-edition/releases/tag/v2024-05-07/ncit.owl",namespace_prefix="NCIT")
snomed = pps2.Resource(id="snomed",name="SNOMED CT",url="https://browser.ihtsdotools.org/",version="https://browser.ihtsdotools.org/?edition=MAIN/2024-12-01",namespace_prefix="SNOMED")
metadata = pps2.MetaData(created = Timestamp(seconds=int(datetime.datetime.now().timestamp())),
                         created_by = "Aldar Cabrelles",
                         submitted_by = "Mickey Mouse",
                         resources=[ncit,snomed],
                         phenopacket_schema_version="2.0")

### Proband

In [5]:
proband = pps2.Individual(id="ID1",sex=MALE)
probandpheno = pps2.Phenopacket(id="ID1_phenopacket",
                                subject = proband,
                                phenotypic_features = [pobese], 
                                diseases = [dthyroid],
                                meta_data = metadata)

### Father

In [6]:
father = pps2.Individual(id="ID2",sex=MALE)
fatherpheno = pps2.Phenopacket(id="ID2_phenopacket",
                               subject = father,
                               phenotypic_features=[pobese],
                               meta_data = metadata)

### Mother

In [7]:
mother = pps2.Individual(id="ID3",sex=FEMALE)
motherpheno = pps2.Phenopacket(id="ID3_phenopacket",
                               subject= mother,
                               diseases = [dthyroid],
                               meta_data = metadata)

## Family

### Pedigree

In [8]:
person1 = Pedigree.Person(
    family_id='FAM001',
    individual_id='ID1',
    paternal_id='ID2',
    maternal_id='ID3',
    affected_status=Pedigree.Person.AFFECTED,
    sex=MALE
)
person2 = Pedigree.Person(
    family_id='FAM002',
    individual_id='ID2',
    paternal_id=None,
    maternal_id=None,
    affected_status=Pedigree.Person.UNAFFECTED,
    sex=MALE
)
person3 = Pedigree.Person(
    family_id='FAM003',
    individual_id='ID3',
    paternal_id=None,
    maternal_id=None,
    affected_status=Pedigree.Person.AFFECTED,
    sex=FEMALE
)
pedigree = Pedigree()
pedigree.persons.append(person1,)
pedigree.persons.append(person2,)
pedigree.persons.append(person3,)

### Joining everything

In [9]:
family = pps2.Family(id="Family",
            proband=probandpheno,
            relatives=[fatherpheno,motherpheno],
            pedigree=pedigree,
            meta_data=metadata)
familyjson = MessageToDict(family)

## Export

In [10]:
with open('/Users/aldar/Documents/CRG-EGA/Curation/Phenopackets/Testing/Output/family.json', 'w', encoding='utf-8') as f:
    json.dump(familyjson, f, ensure_ascii=False, indent=4)